In [1]:
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from scipy.signal.windows import gaussian

In [2]:
# path to the database - change it if needed
path = 'face_data/' 

ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight', 
          'noglasses', 'normal', 'rightlight','sad', 
          'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png

# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)

h = im.shape[0] # hight 
w = im.shape[1] # width

D = h * w
N = len(states)*15 
print(N, D, h, w)

X = np.zeros((D, N))
Y = np.zeros(N)

# collect all data
count = 0 

# there are 15 people
for person_id in range(1, 16):
    for state in states:
        
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        Y[count] = states.index(state)
        
        count += 1 
Y = Y.astype(int)

165 77760 243 320


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Standardize data
X = StandardScaler().fit_transform(X)
# Reduce component
pca = PCA(n_components=125)
X_pca = pca.fit_transform(X.T)

In [4]:
# Split train&test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.3, random_state=1)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def multinomial_logistic_regression(X_train, y_train, X_test, y_test):
    lorg = LogisticRegression(solver='sag', max_iter=3000, penalty='l2')
    lorg.fit(X_train, y_train)
    
    # Prediction
    y_pred_softmax = lorg.predict(X_train)
    y_test_pred_softmax = lorg.predict(X_test)
    
    # Accuracy
    accuracy_train = accuracy_score(y_train, y_pred_softmax)
    accuracy_test = accuracy_score(y_test, y_test_pred_softmax)
    print("Train Accuracy of Multinomial Logistic Regression: ", accuracy_train)
    print("Test Accuracy of Multinomial Logistic Regression: ", accuracy_test)
multinomial_logistic_regression(X_train, y_train, X_test, y_test)

Train Accuracy of Multinomial Logistic Regression:  0.9739130434782609
Test Accuracy of Multinomial Logistic Regression:  0.26


In [6]:
from sklearn.naive_bayes import GaussianNB

def naive_bayes_model(X_train, Y_train, X_test, Y_test):
    gaussian = GaussianNB()
    gaussian.fit(X_train, Y_train)
    
    # Prediction 
    y_pred_gausian = gaussian.predict(X_train)
    y_test_pred_gaussian = gaussian.predict(X_test)
    
    # Accuracy
    accuracy_train = accuracy_score(Y_train, y_pred_gausian)
    print("Train Accuracy on test data of Naive Bayes:", accuracy_train)
    accuracy_test = accuracy_score(Y_test, y_test_pred_gaussian)
    print("Test Accuracy on test data of Naive Bayes:", accuracy_test)
naive_bayes_model(X_train, y_train, X_test, y_test)

Train Accuracy on test data of Naive Bayes: 0.9478260869565217
Test Accuracy on test data of Naive Bayes: 0.12


In [7]:
from sklearn.neural_network import MLPClassifier

def ANN_model(X_train, Y_train, X_test, Y_test):
    ann = MLPClassifier(solver='adam', hidden_layer_sizes=(128, 64), activation='relu', max_iter=3000, random_state=1)
    ann.fit(X_train, Y_train)
    
    # Prediction
    y_pred_ann = ann.predict(X_train)
    y_test_pred_ann = ann.predict(X_test)
    # Accuracy
    accuracy_train = accuracy_score(Y_train, y_pred_ann)
    accuracy = accuracy_score(Y_test, y_test_pred_ann)
    print("Train Accuracy on test data of ANN Classification:", accuracy)
    print("Test Accuracy on test data of ANN Classification:", accuracy)
ANN_model(X_train, y_train, X_test, y_test)

Train Accuracy on test data of ANN Classification: 0.22
Test Accuracy on test data of ANN Classification: 0.22


In [12]:
import cv2
import numpy as np

new_images = []
path_to_new_images = 'new_data/' 
image_filenames = ['pic1.png', 'pic2.png', 'pic3.png', 'pic4.png', 'pic5.png']  

for filename in image_filenames:
    img = cv2.imread(path_to_new_images + filename, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (243, 320))  
    img_flattened = img_resized.flatten() 
    new_images.append(img_flattened)

new_images = np.array(new_images)  

In [9]:
new_images_pca = pca.transform(new_images)

In [10]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)


# Giả sử model là mô hình ANN đã huấn luyện
predictions = gaussian.predict(new_images_pca)
print("Predicted classes for the new images:", predictions)

Predicted classes for the new images: [6 6 6 6 1]
